# 目次
 - はじめに
 - テキスト(csv)ファイル読み込み
 - NetCDFファイル読み込み
 - Fortranバイナリファイル読み込み

## はじめに
データ解析を行う際に、データファイルを読み込むというのは最も基本的な操作になります。  
地球物理学の分野でメジャーなデータファイルの形式は、以下の3つだと思います。  
 1. テキストファイル
 2. NetCDFファイル
 3. Fortranバイナリファイル

Pythonを用いれば、この3つのどの形式のデータファイルも簡単に読み込むことが可能です。  
以下でそれぞれ順を追って説明していきますが、ご自身の必要に応じて2.、3.から読み進める等していただいても構いません。

## テキスト(csv)ファイル読み込み
カンマで区切られているテキストファイルを読み込んでみましょう。  
テキストファイル内の数値データは、numpyのloadtxt関数で読み込むことが出来ます。  
なお、今回読み込むファイルは、[気象庁のページ](http://www.data.jma.go.jp/gmd/cpd/db/elnino/index/nino3irm.html)から取得した、エルニーニョ指数データです。

In [1]:
import numpy as np
data=np.loadtxt('NINO.3.csv',comments='year',delimiter=',',dtype='float')

 - commentsで、読み飛ばす行の左端に存在する文字列を指定する。
 - delimiterで、区切り文字を指定する。もしスペースで区切ってあった場合には、delimiter=...という記述は必要ない。
 - dtypeで、データをどの形式で読み込むかを指定する。デフォルトはfloat(浮動小数点数)。整数として読み込みたい場合にはintとすればよい。

In [2]:
data

array([[  2.00100000e+03,  -4.00000000e-01,  -3.00000000e-01,
         -2.00000000e-01,  -1.00000000e-01,   0.00000000e+00,
          0.00000000e+00,  -1.00000000e-01,  -2.00000000e-01,
         -3.00000000e-01,  -4.00000000e-01,  -5.00000000e-01,
         -4.00000000e-01],
       [  2.00200000e+03,  -3.00000000e-01,  -1.00000000e-01,
          0.00000000e+00,   3.00000000e-01,   4.00000000e-01,
          5.00000000e-01,   6.00000000e-01,   7.00000000e-01,
          8.00000000e-01,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00],
       [  2.00300000e+03,   8.00000000e-01,   5.00000000e-01,
          0.00000000e+00,  -2.00000000e-01,  -2.00000000e-01,
         -2.00000000e-01,  -1.00000000e-01,   1.00000000e-01,
          3.00000000e-01,   4.00000000e-01,   4.00000000e-01,
          4.00000000e-01],
       [  2.00400000e+03,   4.00000000e-01,   3.00000000e-01,
          1.00000000e-01,   0.00000000e+00,  -1.00000000e-01,
          0.00000000e+00,   0.00000000e+00,   2.000

## NetCDFファイル読み込み
netCDFファイルの読み込みには、netCDF4モジュールを用います。  
netCDF4モジュールが使えるようにする方法に関しては[こちら](https://github.com/Unidata/netcdf4-python)をご参照ください。  
試しに、[NOAAのサイト](https://www1.ncdc.noaa.gov/pub/data/cmb/ersst/v4/netcdf/)からダウンロードした2017年2月の全球海面水温データを読み込んでみることにします。  
まず、ターミナル上で  
$ ncdump -c ersst.v4.201702.nc  
と実行して、当該netCDFファイル内のデータの構造に目を通しておくことにします。  

In [3]:
# Jupyter上では、!を先頭につけることでターミナルのコマンドを実行することが出来る。
!ncdump -c ersst.v4.201702.nc

netcdf ersst.v4.201702 {
dimensions:
	lat = 89 ;
	lev = 1 ;
	lon = 180 ;
	time = 1 ;

variables:
	double lat(lat) ;
		lat:units = "degrees_north" ;
		lat:long_name = "Latitude" ;
		lat:standard_name = "latitude" ;
		lat:axis = "Y" ;
		lat:bounds = "lat_bnds" ;
		lat:grids = "Uniform grid from -88 to 88 by 2" ;
	double lev(lev) ;
		lev:units = "meters" ;
		lev:long_name = "Depth of sea surface temperature measurements" ;
		lev:standard_name = "depth" ;
		lev:axis = "Z" ;
		lev:positive = "down" ;
		lev:_CoordinateAxisType = "Height" ;
		lev:comment = "Measurement depth of in situ sea surface temperature varies" ;
	double lon(lon) ;
		lon:units = "degrees_east" ;
		lon:long_name = "Longitude" ;
		lon:standard_name = "longitude" ;
		lon:axis = "X" ;
		lon:bounds = "lon_bnds" ;
		lon:grids = "Uniform grid from 0 to 358 by 2" ;
	float sst(time, lev, lat, lon) ;
		sst:_FillValue = -999.f ;
		sst:long_name = "Extended reconstructed sea surface temperature" ;
		sst:standard_name = "SST" ;
		ss

以上の通り、海面水温データは、'sst'という変数名で格納されているということがわかりました。  
そこで、この海面水温データをnetCDF4モジュールを用いて取り出してみることにします。

In [4]:
import netCDF4
nc=netCDF4.Dataset('ersst.v4.201702.nc','r')
data=nc.variables['sst'][:]

たったこれだけです。  
2行目で、netCDFファイルの中身の情報を持った「netCDF4オブジェクト」を生成し、  
3行目で、そのオブジェクトの一要素(今回の場合'sst'という変数名で格納されたデータの中身)を取り出しているわけです。

In [5]:
print(type(nc))
print(data)

<class 'netCDF4._netCDF4.Dataset'>
[[[[-- -- -- ..., -- -- --]
   [-- -- -- ..., -- -- --]
   [-- -- -- ..., -- -- --]
   ..., 
   [-1.5788253545761108 -1.5387723445892334 -1.518031358718872 ...,
    -1.7999999523162842 -1.7702364921569824 -1.7328710556030273]
   [-1.5536582469940186 -1.5258724689483643 -1.5181310176849365 ...,
    -1.6532748937606812 -1.6176127195358276 -1.5812113285064697]
   [-1.7319483757019043 -1.7999999523162842 -1.7999999523162842 ...,
    -1.741258144378662 -1.7337182760238647 -1.729689598083496]]]]


## Fortranバイナリファイル読み込み
ここでは、まず  
**リトルエンディアンのヘッダなし4バイト浮動小数点バイナリ(俗に言うGrADS形式と呼ばれるもの)**  
の中身を読み込んでみることにします。  

In [6]:
!cat write_binary.f90 

program main

  implicit none

  integer,parameter::N=10,M=20
  integer::i,j
  real,dimension(1:N,1:M)::x

  open(10,file='filename.out',form='unformatted',access='direct',recl=N*4)

  do i = 1,N
     do j = 1,M
        x(i,j) = i+j*2
     end do
  end do

  do j = 1,M
     write(10,rec=j)(x(i,j),i=1,N)
  end do

  close(10)

end program main


まず、fortranプログラムを実行して架空のデータを作成。

In [7]:
!gfortran write_binary.f90

In [8]:
!./a.out

続いてpythonでファイルの読み込みです。

In [9]:
import numpy as np
N=10  #1レコード番号あたりに格納されているデータの数。
M=20  #レコードの総数。
f=open('filename.out','r')
dty=np.dtype([('data','<'+str(N)+'f')])
chunk=np.fromfile(f,dtype=dty,count=M)
data=np.array([chunk[j]['data'] for j in range(0,M)])

In [10]:
data

array([[  3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.],
       [  5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.],
       [  7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.],
       [  9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.],
       [ 11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.],
       [ 13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.],
       [ 15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.],
       [ 17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.],
       [ 19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.],
       [ 21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.],
       [ 23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.],
       [ 25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.],
       [ 27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.],
       [ 29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.],
       [ 31.,  32.,  33.,  34.,  3

最終行は、  


In [11]:
for j in range(0,M):  
     data[j,:]=chunk[j]['data']

を1行で書き換えたものです。  
chunk[k-1]が、fortranで言うところのレコード番号kのデータに対応しています。  
なので、例えばレコード番号6のデータだけを取り出したい場合には、最終行を  

In [12]:
data=chunk[5]['data']  

に置き換えれば良いです。

In [13]:
data

array([ 13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.], dtype=float32)